In [1]:
import sys
sys.path.append("..")
from qtn.maxkappa import MaxKappa
from qtn.bimax import BiMax
from qtn.util import (boltzmann, emass, echarge, permittivity, cspeed)
import mpmath as mp
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from scipy.io.idl import readsav

/tmp/ipykernel_3901960/2188330759.py:11: DeprecationWarning: Please use `readsav` from the `scipy.io` namespace, the `scipy.io.idl` namespace is deprecated.
  from scipy.io.idl import readsav


###### load electron/proton parameters and TNR data

In [2]:
p_dat=readsav('/Users/tongyuguang/Google Drive/research/new_data/idlsave_param_electron_19970111.sav')
m_dat=readsav('/Users/tongyuguang/Google Drive/research/data/meudon_tnr/TNR_XY_ACE_19970111.sav')
m_data=m_dat['data']
mt = m_data['timeur8'][0]
mtag = m_data['time'][0]

FileNotFoundError: [Errno 2] No such file or directory: '/Users/tongyuguang/Google Drive/research/new_data/idlsave_param_electron_19970111.sav'

In [ ]:
plt.plot(p_dat['tc'])
plt.show()

###### WIND WAVES specification

In [ ]:
ant_len = 50      # m (monopole) 
ant_rad = 1.9e-4  # m
base_cap = 20e-12 # Fara
fbins = np.array([4000*2**((2*i+1)/32) for i in range(96)])

### example

In [ ]:
i=280
print(p_dat['time_in_string'][i])
print(p_dat['tc'][i])

In [ ]:
vsw = p_dat['vsw'][i] * 1000
tp = p_dat['tp'][i]
tc = p_dat['tc'][i]
th = p_dat['th'][i]
t = th/tc
nc = p_dat['nc'][i]
print(p_dat['nh'][i])
nh = p_dat['netnr'][i]-nc
n = nh/nc
ne = nc + nh
k = float(p_dat['kh'][i])
fpe = np.sqrt(echarge**2 * ne * 1e6 /emass/permittivity)/2/np.pi
print([tc, tp, nc, nh, fpe])
print([ne, n, t, tp, tc, k, vsw])

In [ ]:
p_dat.keys()

In [ ]:
netnr = p_dat['netnr'][i]
print(netnr)

In [ ]:
p = MaxKappa(ant_len, ant_rad, base_cap)

In [ ]:
za_zr = np.array([np.array(p.impedance(f, ne, n, t, tp, tc, k, vsw)) for f in fbins])

In [ ]:
za = za_zr[:, 0]
zr = za_zr[:, 1]

In [ ]:
gain = np.abs((za+zr)/zr)**2

In [ ]:
plt.plot(gain)
plt.show()

In [ ]:
def helper(f):
    if f < fpe or f > 1.2 * fpe:
        mp.mp.dps = 20
    else:
        mp.mp.dps = 40
    val = p.electron_noise(f, ne, n, t, tp, tc, k, vsw)
    mp.mp.dps= 15
    return val

In [ ]:
e_noise = np.array([helper(f) for f in fbins])

In [ ]:
p_noise = np.array([p.proton(f, ne, n, t, tp, tc, k, vsw) for f in fbins])

In [ ]:
def shot(za, sc_pot):
    _vtc = np.sqrt(2 * echarge * tc/emass)
    _nc = nc * 1e6
    _ne = _nc * _vtc * (1 + n * np.sqrt(t)) * 2 * np.pi * ant_rad * ant_len / np.sqrt(4 * np.pi)
    A = 1 + echarge * sc_pot/tc
    return 2 * A * echarge**2 * np.abs(za)**2 * _ne 

In [ ]:
s_noise = np.array([shot(_za, 0) for _za in za])

In [ ]:
jj = 15020
time_tag = mtag[jj]
print(time_tag)

In [ ]:
spec = m_data['spectra'][0][:, jj]
spec = 10. ** (spec/10.)

In [ ]:
plt.plot(fbins, spec)
plt.xscale('log')
plt.yscale('log')
plt.show()

In [ ]:
plt.rc('text', usetex=False)
plt.rc('font', family='serif')
fig = plt.figure(figsize=[6, 6])
plt.plot(fbins/1000*np.sqrt(netnr/ne), (e_noise + p_noise + s_noise)/gain, label='theory/3dp fits')
plt.plot(fbins/1000, spec, 'o', markersize=2, label='WIND TNR')
plt.plot(fbins/1000, e_noise/gain,'--', label='electron')
plt.plot(fbins/1000, p_noise/gain, '-.', label='proton')
plt.plot(fbins/1000, s_noise/gain, '--', label='shot')
plt.xscale('log')
plt.yscale('log')
plt.xlim([4, 256])
#plt.ylim([1e-18, 1e-10])
plt.xlabel(r'$f[kHz]$')
plt.ylabel(r'$V_r^2[V^2Hz^{-1}]$')
plt.title(time_tag.decode("utf-8"))
plt.legend(loc='best')
plt.show()

In [ ]:
fig.savefig('wind1.png', dpi=300)